# 학업탄력성 영향요인 연구
@author: sjh

## 1. Load

In [1]:
import os
import copy
import json
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sys import platform

import warnings
warnings.filterwarnings('ignore')

# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.autolayout"] = True

# 설치된 폰트 출력
import matplotlib.font_manager as fm
font_list = [font.name for font in fm.fontManager.ttflist]
if 'darwin' in platform:
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'Malgun Gothic'


BASE_DIR = os.getcwd()
print('>> Current OS: ', platform)
print('>> Current WD: ', BASE_DIR)

>> Current OS:  win32
>> Current WD:  c:\Users\jhun1\Proj\Research\MixedRF


In [2]:
class Load:
    def __init__(self, stuFolder, schFolder, tchFolder, codeBook,
                onlyCodeBook, ):
        self.BASE_DIR = r'C:\Users\jhun1\Dropbox\[3]Project\[혼합효과 랜덤포레스트_2022]\PISA2018'

        if onlyCodeBook == False:
            print('>>>>> Init: load raw data')
            stuFile = [FILE for FILE in os.listdir(os.path.join(self.BASE_DIR, stuFolder)) if FILE[-4:] == '.sav'][0]
            schFile = [FILE for FILE in os.listdir(os.path.join(self.BASE_DIR, schFolder)) if FILE[-4:] == '.sav'][0]
            tchFile = [FILE for FILE in os.listdir(os.path.join(self.BASE_DIR, tchFolder)) if FILE[-4:] == '.sav'][0]

            self.rawStu = pd.read_spss(os.path.join(self.BASE_DIR, stuFolder, stuFile))
            self.rawSCH = pd.read_spss(os.path.join(self.BASE_DIR, schFolder, schFile))
            self.rawTCH = pd.read_spss(os.path.join(self.BASE_DIR, tchFolder, tchFile))
            self.dataLS = [self.rawStu, self.rawSCH, self.rawTCH]

            # desciptive
            print('>> Stu data set', self.rawStu.shape)
            print('>> Sch data set', self.rawSCH.shape)
            print('>> Tch data set', self.rawTCH.shape)
        
        else:
            print('>> Only Codebook will be loaded')
            pass
        
        self.cb = pd.read_excel(codeBook)



    def defaultCleaner(self):
        print('\n\n>>>> Cleaning: default nation and variable')


        ### SK, US dict 형태 잡아줌
        def cleaningNational(dataLS, SouthKorea = 'Korea', US='United States'):
            nationalData = {'SK': [], 'US': []}

            for nation_name, code in zip(nationalData.keys(), [SouthKorea, US]):
                print(f'\n>> slicing: {nation_name}')
                for data in dataLS:
                    # print(data.head(5))
                    temp2 = data[data['CNTRYID'] == code]
                
                    nationalData[nation_name].append(temp2)
                    print('>> sliced shape: ', temp2.shape)
            
            return nationalData
            

        def cleaningVariable(data, codeBook):
            output = {}
            
            for nation_name in data.keys():
                Column_toSave = {'Stu': [], 'Sch': [], 'Tch': []}
                for fileName, variable, category in tqdm(zip(codeBook['file name'].values, codeBook['NAME'].values, codeBook['categories']), desc=">> variable check"):
                    # 코드북 내에서도 분석에서 제할 변수는 file name을 비움
                    if type(fileName) != str:
                        continue


                    else: 
                        if category == 'identifier':
                            Column_toSave['Stu'].append(variable)
                            
                            if variable == 'CNTSTUID':
                                continue
                            else: # 학교, 교사 셋에는 해당 변수가 없어서 추가해줌
                                Column_toSave['Sch'].append(variable)
                                Column_toSave['Tch'].append(variable)

                        else:
                            if 'STU' in fileName:
                                if variable in data[nation_name][0].columns:
                                    Column_toSave['Stu'].append(variable)
                                else:
                                    print('>> none(stu)', variable)

                            elif 'SCH' in fileName:
                                if variable in data[nation_name][1].columns:
                                    Column_toSave['Sch'].append(variable)
                                else:
                                    print('>> none(sch)', variable)

                            elif 'TCH' in fileName:
                                if variable in data[nation_name][2].columns:
                                    Column_toSave['Tch'].append(variable)
                                else:
                                    print('>> none(tch)', variable)
                    
                
                # print('>>>> save: ', Column_toSave)
                output[nation_name] = [data[nation_name][0][Column_toSave['Stu']],
                                        data[nation_name][1][Column_toSave['Sch']],
                                        data[nation_name][2][Column_toSave['Tch']],
                                        ]
                assert len(Column_toSave['Tch']) < 4, print(Column_toSave['Tch'])

            assert 'SK' in output.keys()
            assert 'US' in output.keys()
            return output

        cleaned_Nation = cleaningNational(dataLS = self.dataLS)
        self.default_cleaningData = cleaningVariable(data = cleaned_Nation, codeBook = self.cb)

        # categories 에서 일단은 codebook에 있는 변수가 다 있나 확인
        # categories 에서 individual & family, school 구분



LOAD_ONLY_CODEBOOK = True

Loader = Load(stuFolder="STU", schFolder='SCH', tchFolder='TCH',
                onlyCodeBook = LOAD_ONLY_CODEBOOK,
                codeBook=r'C:\Users\jhun1\Dropbox\[3]Project\[혼합효과 랜덤포레스트_2022]\drive-download-20220816T053902Z-001\PISA2018_CODEBOOK (변수선택-공유).xlsx'
                )
if LOAD_ONLY_CODEBOOK == False:
    Loader.defaultCleaner()

>> Only Codebook will be loaded


In [ ]:
Loader.default_cleaningData.keys()

In [ ]:
with pd.ExcelWriter(os.path.join(BASE_DIR, 'data', 'cleanedData(SK).xlsx')) as writer:
    Loader.default_cleaningData['SK'][0].to_excel(writer, sheet_name='stu', index=False)
    Loader.default_cleaningData['SK'][1].to_excel(writer, sheet_name='sch', index=False)
    Loader.default_cleaningData['SK'][2].to_excel(writer, sheet_name='tch', index=False) 


with pd.ExcelWriter(os.path.join(BASE_DIR, 'data', 'cleanedData(US).xlsx')) as writer:
    Loader.default_cleaningData['US'][0].to_excel(writer, sheet_name='stu', index=False)
    Loader.default_cleaningData['US'][1].to_excel(writer, sheet_name='sch', index=False)
    Loader.default_cleaningData['US'][2].to_excel(writer, sheet_name='tch', index=False)

## 2. Preprocessing

In [ ]:
# 앞선 처리 데이터 불러오기
loadedData = {'SK': [
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='stu'),
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='sch'),
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(SK).xlsx'), sheet_name='tch'),
            
            ],
        'US': [
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='stu'),
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='sch'),
            pd.read_excel(os.path.join(BASE_DIR,'data', 'cleanedData(US).xlsx'), sheet_name='tch'),
            ]}

In [ ]:
class Preprocessing:
    def __init__(self, LoadedData, codeBook, dummyCodeBook):
        self.data = LoadedData
        self.cb = codeBook
        with open(dummyCodeBook, encoding='utf-8') as json_file:
            self.dummyCB = json.load(json_file)
        self.testBook = {
                    'read/math/sci_1': 'PV1MATH PV1READ PV1SCIE'.split(),
                    'read/math_1': 'PV1MATH PV1READ'.split(),
                    'read/math_10': 'PV1MATH PV2MATH PV3MATH PV4MATH PV5MATH PV6MATH PV7MATH PV8MATH PV9MATH PV10MATH PV1READ PV2READ PV3READ PV4READ PV5READ PV6READ PV7READ PV8READ PV9READ PV10READ'.split(),
                    'read_1': ['PV1READ']
                }

        self._1_dummy = {}
        self._2_joined = {}
        self._3_dropNa = {}
        self._4_ESCS = {'full': {}, 'sliced': {}} # 여기서 데이터 갈라야함
        self._5_shouldBeCal = {}
        self.finalRS = {}
        
        self.rs_1_columnFull = pd.DataFrame()
        self.rs_1_columnSK = pd.DataFrame()
        self.rs_1_columnUS = pd.DataFrame()
        

    ### Needs1. is selected variable contained in both dataset
    def noDataColumn(self): # 열별로 계산
        toDrop = {}
        
        for nationName, nationalData in self.data.items():
            print('>> test NA column: ', nationName)

            toDrop[nationName] = []
            for idx, (label, inputDf) in enumerate(zip('stu sch tch'.split(), nationalData)):
                if label == 'tch':
                    continue
                else:
                    print(label)
                    for column in inputDf.columns:
                        if inputDf[column].isna().sum() > (inputDf.shape[0] * 0.8):
                            print('>>> over 80% is NA: ', column)
                            toDrop[nationName].append(column)
                        
                        elif 'missing' in inputDf[column].values:
                            print('>>> missing: ', column)
                            toDrop[nationName].append(column)
                        
                        else:
                            continue
        return toDrop
            # assert len(toDrop[nationName]) == 2, print(toDrop)


    def Dummy(self, doDummy):
        # match key and value from codeBook
        print('\n\n>>>> 1. Dummy coding')
        def matchKV(codeBookDict, inputList):
            outputLS = []
            for val in inputList:
                try:
                    outputLS.append(codeBookDict[val])
                except KeyError:
                    outputLS.append(np.nan)
            
            return outputLS
        
        if doDummy == True:
            notDummyCol1 = self.cb[self.cb['categories'] == 'identifier'].index
            notDummyCol2 = self.cb[self.cb['categories'] == 'resilient status'].index
            notDummyCol3 = self.cb[self.cb['file name'] == 'should be caculated'].index
            
            toDummy = self.cb.drop(list(notDummyCol1)+list(notDummyCol2)+list(notDummyCol3), axis=0) # 더미 변환 안할 변수 행 삭제함
            # display(toDummy)

            for nationalName, inputNational in self.data.items():
                outputNational = copy.deepcopy(inputNational)

                for idx, row in toDummy.iterrows(): # 변수별로 반복문
                    variable = row['NAME']
                    
                    if type(row['file name']) != str: # 분석에서 제외할 변수가 있어서 버림
                        continue

                    else:
                        if ('STU' in row['file name']) and (variable in self.dummyCB['stu']):
                            outputLS = matchKV(self.dummyCB['stu'][variable], outputNational[0][variable])
                            outputNational[0][variable] = outputLS 

                    # 학교, 교사 데이터는 더미코딩할 것 없음
                self._1_dummy[nationalName] = outputNational
        elif doDummy == False:
            self._1_dummy = copy.deepcopy(self.data)
        else:
            raise TypeError('>> Error: check option Type')


    def Join(self):
        print('\n\n>>>> 2. Join DataFrame')

        for nationalName, inputNational in self._1_dummy.items():
            print('>> join nation: ', nationalName)
            inputNational[0].reset_index(drop=True, inplace=True)
            
            outputDf = copy.deepcopy(inputNational[0])
            # print('>> before ', outputDf.shape)
            before = outputDf.shape

            # if sampler.US[1].index.name == 'CNTSCHID':
            inputNational[1].drop(['CNTRYID', 'CNT'], axis=1, inplace=True)
            if inputNational[1].index.name == 'CNTSCHID':
                pass
            else:
                inputNational[1].set_index('CNTSCHID', drop=True, inplace=True)
            
            # print(inputNational[1].index)
            for idx, row in tqdm(outputDf.iterrows(), desc=">> mapping"):
                toBeInput = inputNational[1].loc[row['CNTSCHID']].values # 학생 데이터에 들어가야할 학교 데이터 찾기
                assert len(toBeInput) == inputNational[1].shape[1]
                
                toBeInput_T = toBeInput.reshape(1, 8)
                outputDf.loc[idx, list(inputNational[1].columns)] = toBeInput_T[0]
            
            # print('>> after ', outputD>f.shape)
            after = outputDf.shape
            print('>>>> Bef: ', before, '....', 'Aft: ', after)
            assert 'EDUSHORT' in outputDf.columns

            self._2_joined[nationalName] = outputDf

    def DropStudent(self):
        # 각 column 별로 데이터 검수
        def column_wise(inputData):
            if type(inputData) == dict:
                merged = pd.concat([inputData['SK'], inputData['US']])
                assert merged.shape[0] == inputData['SK'].shape[0] + inputData['US'].shape[0]
            elif type(inputData) == pd.DataFrame:
                merged = copy.deepcopy(inputData)
            
            else:
                raise TypeError('>> Error: Check your input D type')
                

            describeDF = merged.describe().T
            describeDF['NA_ratio'] = round(
                100 - describeDF['count']/merged.shape[0]*100,
                 2
                 )

            newColumnOrder = [describeDF.columns[0], 'NA_ratio'] + list(describeDF.columns[1:-1])
            describeDF= describeDF[newColumnOrder]
            return describeDF

        #!# 각 학생별로 데이터 검수
        def row_wise(inputData):
            merged = pd.concat([inputData['SK'], inputData['US']])
            assert merged.shape[0] == inputData['SK'].shape[0] + inputData['US'].shape[0]
            # unlike column wise, we prepare data with 

            for_histogram = {}
            for label, data in zip(['full', 'SK', 'US'], [merged, inputData['SK'], inputData['US']]):
                for_histogram[label] = []

                for i in range(len(data.index)) :
                    for_histogram[label].append(data.iloc[i].isnull().sum())

            fig = plt.figure(figsize=(17,6))

            plt.subplot(1, 3, 1)
            plt.hist(for_histogram['full'])
            plt.title('\n전체 데이터\n')
            plt.xlabel('\n학생별 결측비율\n')
            
            
            plt.subplot(1, 3, 2)
            plt.hist(for_histogram['SK'])
            plt.title('\nSouth Korea\n')
            plt.xlabel('\n학생별 결측비율\n')
            
            plt.subplot(1, 3, 3)
            plt.hist(for_histogram['US'])
            plt.title('\nUnited States\n')
            plt.xlabel('\n학생별 결측비율\n')

            plt.savefig(os.path.join(BASE_DIR, 'data', f'NA_ratio.jpg'))
            plt.show()
        
        def dropOver(inputData):
            output = copy.deepcopy(inputData)
            return output
        
        self.rs_1_columnFull = column_wise(self._2_joined)
        self.rs_1_columnSK = column_wise(self._2_joined['SK'])
        self.rs_1_columnUS = column_wise(self._2_joined['US'])

        row_wise(self._2_joined)
        self._3_dropNa = dropOver(self._2_joined)
        
    def ESCS(self):
        #!# 데이터를 두개 만들어야함, result, resultSlice
        print('\n\n>>>> 4. Slicing data by ESCS')
        def escsSlice(inputDict):
            assert type(inputDict) == dict, print('>> Error: must input Dict')
            output = {'SK': pd.DataFrame(), 'US': pd.DataFrame()}
            for nationalName, inputNational in inputDict.items():
                slicedData = copy.deepcopy(inputNational)
                escsVar = slicedData['ESCS'].quantile(0.25)
                
                before = slicedData.shape[0]
                toDrop = []
                for idx, val in enumerate(slicedData['ESCS'].values):
                    if val < escsVar:
                        continue
                    else:
                        toDrop.append(idx) # escs 하위 25%를 넘는 친구들은 버림
                
                slicedData.drop(toDrop, axis=0, inplace=True)
                output[nationalName] = slicedData
                after = slicedData.shape[0]
                print('>> before: ', before, '>> after: ', after)
            
            return output

        def quantileCalculator( 
                            inputData, # 전체 Full, escs 하위 25%로 데이터셋이 2개로 나뉘므로 인풋을 줘야함
                            targetColumn, # 어떻게 변수를 넣어서 테스트할 것인지
                            figName, # 그림 제목
                            option, # full: 전체 데이터, sliced: 잘린 데이터
                            ):
            

            output = {'SK': pd.DataFrame(), 'US': pd.DataFrame()}
            for IDX, (nationalName, inputNational) in enumerate(inputData.items()):
                
                inputNational['AcademicScore'] = inputNational.loc[:, targetColumn].mean(axis=1)
                acadVar = inputNational['AcademicScore'].quantile(0.75) # 상위 25%
                if option == 'full':
                    escsVar = inputNational['ESCS'].quantile(0.25) # 하위 25%
                elif option == 'sliced':
                    escsVar = inputNational['ESCS'].quantile(0.0) # 단순히 avgline 그릴 때 방해 안되려고 넣은 것임
                total = inputNational.shape[0]
                
                iamResilient = []
                if option == 'full':
                    for idx, row in inputNational.iterrows():
                        if row['AcademicScore'] > acadVar and row['ESCS'] < escsVar: # sliced 데이터에서는 이 기준을 만족할 수 없음
                            iamResilient.append(1)
                        else:
                            iamResilient.append(0)
                elif option == 'sliced':
                    for idx, row in inputNational.iterrows():
                        if row['AcademicScore'] > acadVar: # sliced 데이터는 escs 기준 필요 없음
                            iamResilient.append(1)
                        else:
                            iamResilient.append(0)

                inputNational['resilient'] = iamResilient
                resilientCount = [x for x in iamResilient if x ==1]
                print(f'\n>> 회복탄력성 학생수({nationalName}): ', len(resilientCount), f'({round(len(resilientCount)/total*100, 2)})%')
                fig = plt.figure(figsize=(17,9))

                plt.subplot(2, 2, 2*IDX+1)
                plt.hist(inputNational['AcademicScore'])
                plt.title(f'\n학업성취{nationalName}\n')
                plt.xlabel('\n점수\n')
                plt.axvline(acadVar, color='r', linewidth=1, linestyle='--')
                
                plt.subplot(2, 2, 2*IDX+2)
                plt.hist(inputNational['ESCS'])
                plt.title(f'\n사회문화경제{nationalName}\n')
                plt.xlabel('\n점수\n')
                plt.axvline(escsVar, color='r', linewidth=1, linestyle='--')

                output[nationalName] = inputNational
            
            plt.savefig(os.path.join(BASE_DIR, 'data', f'{figName}.jpg'))
            plt.show()

            return output
        
        self._4_ESCS['full'] = copy.deepcopy(self._3_dropNa) # no drop case, so just copied
        self._4_ESCS['sliced'] = escsSlice(self._3_dropNa)

        assert type(self._4_ESCS['full']) == dict, print(self._4_ESCS['full'])
        # print(self._4_ESCS['full'].keys())

        self._4_ESCS['full'] = quantileCalculator(inputData=self._4_ESCS['full'], 
                                                targetColumn=self.testBook['read_1'], 
                                                figName ='읽1',
                                                option = 'full')
        self._4_ESCS['sliced'] = quantileCalculator(inputData=self._4_ESCS['sliced'], 
                                                targetColumn=self.testBook['read_1'], 
                                                figName ='읽1(target paper)', 
                                                option = 'sliced')

    
    # should be calculated 변수들 계산하는 것임
    def shouldBeCalculated(self):
        print('\n\n>>>> 6. Should Be Calculated')
        
        def schoolMean(inputDf, whichVar):
            assert type(whichVar) == list
            outputMean = {}
            for sch_id in inputDf['CNTSCHID'].values:
                # print('>> ', sch_id)
                if sch_id in outputMean.keys():
                    continue
                
                else:
                    temp1 = inputDf[inputDf['CNTSCHID'] == sch_id]
                    temp2 = temp1.loc[:, whichVar] 
                    assert len(temp2.columns) == len(whichVar)
                    meanVal = np.nanmean(temp2.values)
                    assert type(meanVal) == np.float64, print('Error : ', type(meanVal))

                    outputMean[sch_id] = meanVal
            
            return outputMean
        
        
        def meanMapping(inputColumn, mean_dict):
            outputLS = []
            for idx, sch_id in enumerate(inputColumn.values):
                outputLS.append(mean_dict[sch_id])

            return outputLS
        

        def matching(inputData, codeBook):
            output = copy.deepcopy(inputData)
            shouldBeCal = self.cb[self.cb['file name'] == 'should be caculated']
            assert len(shouldBeCal) == 2, print('Error: check self.cb')
            for national in output.keys():
                beforeShape = output[national].shape[1]
                
                calVal = list(codeBook[codeBook['categories'] == 'resilient status']['NAME'])
                calVal.remove('ESCS')
                
                for variable in shouldBeCal['NAME'].values:
                
                    if variable == 'AVG_S_TEST':    
                        mean_dict = schoolMean(output[national], calVal)
                        # print(f'>> {variable} len: ', len(mean_dict.keys()))
                        
                    elif variable == 'AVG_S_ESCS':
                        mean_dict = schoolMean(output[national], ['ESCS'])
                        # print(f'>> {variable} len: ', len(mean_dict.keys()))

                    #평균 dict 활용해서 매칭 진행
                    outputLS = meanMapping(output[national]['CNTSCHID'], mean_dict)
                    assert len(outputLS) == output[national].shape[0], print('Error: ', len(outputLS))
                    output[national][variable] = outputLS # 학교 데이터이므로, 학교에 맞춰서 추가하기

                afterShape = output[national].shape[1]
                assert afterShape - beforeShape == 2, print('Beofre: ', beforeShape, ' ... ', 'After: ', afterShape)
            
            return output

        self._5_shouldBeCal['full'] = matching(self._4_ESCS['full'], codeBook = self.cb)
        self._5_shouldBeCal['sliced'] = matching(self._4_ESCS['sliced'], codeBook = self.cb)

    
    def AdjustMinor(self):
    # ```
    # 마이너한 것들을 조정하기 위함
    # ```
        print('\n\n>>>> 7. Manually adjust')
        # 두 데이터를 나라 row를 만들고, 합치기 위함
        def Merge(inputData):
            output = pd.concat([inputData['SK'], inputData['US']], axis=0)
            assert inputData['SK'].shape[0] + inputData['US'].shape[0] == output.shape[0]

            dropAcademic = ['CNTRYID', 'AcademicScore']
            for column in output.columns:
                if 'PV' in column:
                    dropAcademic.append(column)

            output.drop(dropAcademic, axis=1, inplace=True)
            # print('>> columns: ', output.columns)
            return output

        # spss 편하도록, 주요 변수들을 앞으로 빼는 작업
        def columnOrder(inputData,
                        important_columns=['resilient']):
            column_ID = ['CNT', 'CNTSCHID', 'CNTSTUID']
            # try1
            # inputData.set_index(important_columns, inplace=True)
            # print('1. ', inputData.columns)

            # idxDf = inputData.pop(column_ID)
            # inputData.reset_index(inplace=True)
            # output = inputData.insert(0, idxDf.name, idxDf)
            # print('2. ', output.columns) 


            inputData.set_index(column_ID+important_columns, inplace=True)
            # print(inputData.columns)
            inputData.reset_index(inplace=True)
            # print(inputData.columns)

            return inputData

        self.finalRS['full'] = Merge(self._5_shouldBeCal['full'])
        self.finalRS['sliced'] = Merge(self._5_shouldBeCal['sliced'])

        self.finalRS['full'] = columnOrder(self.finalRS['full'])
        self.finalRS['sliced'] = columnOrder(self.finalRS['sliced'])


processor = Preprocessing(LoadedData=loadedData, codeBook=Loader.cb, dummyCodeBook='dummyCoding.json')
processor.noDataColumn()
processor.Dummy(doDummy=False) # 굳이 dummy할 필요가 없음, rf에서 categorical / numerical 인식해야함
processor.Join()
processor.DropStudent()
processor.ESCS()
processor.shouldBeCalculated()
processor.AdjustMinor()

In [ ]:
with pd.ExcelWriter(os.path.join(BASE_DIR, 'data', 'preprocessing.xlsx')) as writer:
    processor.finalRS['full'].to_excel(writer, sheet_name='full', index=False)
    processor.finalRS['sliced'].to_excel(writer, sheet_name='sliced', index=False)

with pd.ExcelWriter(os.path.join(BASE_DIR, 'data', 'descriptive(raw).xlsx')) as writer:
    processor.rs_1_columnFull.to_excel(writer, sheet_name='dsec_full', index=True)
    processor.rs_1_columnSK.to_excel(writer, sheet_name='dsec_SK', index=True)
    processor.rs_1_columnUS.to_excel(writer, sheet_name='dsec_US', index=True)

## Descriptive Statics

In [3]:
# 앞선 처리 데이터 불러오기
cleanedData = pd.read_excel(os.path.join(BASE_DIR, 'data', 'preprocessing.xlsx'))

In [14]:
class Descriptive:
    def __init__(self, cleanedData):
        self.df = cleanedData
        self.NUM = {'full': pd.DataFrame(), 'SK': pd.DataFrame(), 'US': pd.DataFrame()}
        self.CAT = {'full': pd.DataFrame(), 'SK': pd.DataFrame(), 'US': pd.DataFrame()}

        self._1_full = pd.DataFrame()
        self._2_SK = pd.DataFrame()
        self._3_US = pd.DataFrame()

    def Numeric(self):

        def naRatio(fullCount, inputArr):
            output = 100 - inputArr/fullCount*100
            return np.round(output, decimals=2)

        def Full(inputData):
            output = inputData.describe().T.round(2)
            output['na_ratio'] = naRatio(inputData.shape[0], output['count'])
            return output

        def perNation(inputData):
            # print(inputData['CNT'].unique())

            output= {}
            for nation in inputData['CNT'].unique():
                df_nation = inputData[inputData['CNT'] == nation]
                output[nation]= df_nation.describe().T.round(2)
                output[nation]['na_ratio'] = naRatio(df_nation.shape[0], output[nation]['count'])

            return output

        self.NUM['full'] = Full(self.df)
        nationDict = perNation(self.df)
        self.NUM['SK'] = nationDict['Korea']
        self.NUM['US'] = nationDict['United States']

    def Categorical(self):

        def sliceCategorical(inputData,
                            referenceNumerical # for assert 
                            ):   
            num_cols = inputData._get_numeric_data().columns
            cat_cols = list(set(inputData.columns) - set(num_cols))
            
            assert list(num_cols) == list(referenceNumerical.index), print(num_cols)

            return cat_cols

        def Full(inputData, cat_cols):
            df_cat = inputData[cat_cols]
            temp = df_cat.describe(include='all').T
            temp['na_ratio'] = temp['freq'].values/temp['count'].values*100
            # temp['na_ratio'] = np.round(temp['na_ratio'], 2) #!# error: infinite
            return temp
            

        def perNation(inputData, cat_cols):
            output= {}
            for nation in inputData['CNT'].unique():
                df_nation = inputData[inputData['CNT'] == nation]
                df_cat = df_nation[cat_cols]
                temp = df_cat.describe(include='all').T
                
                temp['na_ratio'] = temp['freq'].values/temp['count'].values*100
                # temp['na_ratio'] = np.round(temp['na_ratio'], 2) #!# error: infinite
                output[nation] = temp.round(2)
                
            return output

        
        cat_cols = sliceCategorical(self.df, referenceNumerical=self.NUM['full'])
        self.CAT['full'] = Full(self.df, cat_cols=cat_cols)
        nationDict = perNation(self.df, cat_cols=cat_cols)
        self.CAT['SK'] = nationDict['Korea']
        self.CAT['US'] = nationDict['United States']
        
    
    def Join(self):

        def merger(df_num, df_cat):
            output = pd.concat([df_num, df_cat], axis=0) # concate on row
            # for cat_idx in df_cat.index:
            # 합치면 좋겠지만, num, cat 구분 잘되서 좋음

            ## drop unnecessary columns
            output.drop(columns=['min', '25%', '50%', '75%', 'max'], inplace=True)
            output.drop(index=['CNTSCHID', 'CNTSTUID', 'CNT'], inplace=True)

            ## re-order columns 
            output.reset_index(drop=False, inplace=True)
            output.set_index(['index', 'count', 'na_ratio'], inplace=True)
            output.reset_index(drop=False, inplace=True)
            output.set_index('index',inplace=True)
            return output.round(2)
        
        self._1_full = merger(self.NUM['full'], self.CAT['full'])
        self._2_SK = merger(self.NUM['SK'], self.CAT['SK'])
        self._3_US = merger(self.NUM['US'], self.CAT['US'])

        # display(self._1_full)
        # display(self._2_SK)
        # display(self._3_US)
        

analyzer = Descriptive(cleanedData)
analyzer.Numeric()
analyzer.Categorical()
analyzer.Join()

In [15]:
with pd.ExcelWriter(os.path.join(BASE_DIR, 'data', 'descriptive(cleaned).xlsx')) as writer:
    analyzer._1_full.to_excel(writer, sheet_name='full', index=True)
    analyzer._2_SK.to_excel(writer, sheet_name='Korea', index=True)
    analyzer._3_US.to_excel(writer, sheet_name='United States', index=True)

# Random Forest
- just for comparing results of random forest btw python and r